# Faster R-CNN Model

In [2]:
# Importing

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import json
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
n_epochs = 200
minibatch_size = 50
lr = 1e-4
keep = 0.5

In [4]:
x = tf.placeholder(tf.float32, shape = [None, 28,28,1]) #shape in CNNs is always None x height x width x color channels
y_ = tf.placeholder(tf.float32, shape = [None, 10]) #shape is always None x number of classes

In [5]:
def weight_variable(shape):
    """Initializes weights randomly from a normal distribution
    Params: shape: list of dimensionality of the tensor to be initialized
    """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Initializes the bias term randomly from a normal distribution.
    Params: shape: list of dimensionality for the bias term.
    """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    """Performs a convolution over a given patch x with some filter W.
    Uses a stride of length 1 and SAME padding (padded with zeros at the edges)
    Params:
    x: tensor: the image to be convolved over
    W: the kernel (tensor) with which to convolve.
    """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# strides is a length-4 list that specifies the amount to move for each dimension of our input x. 
# the dimensions correspond to the following (in order): batch_size, length of image, width of image, # of channels in image

def max_pool_2x2(x):
    """Performs a max pooling operation over a 2 x 2 region"""
    # ksize: we only want to take the maximum over 1 example and 1 channel. 
    # the middle elements are 2 x 2 because we want to take maxima over 2 x 2 regions
    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME') # stride 2 right and 2 down

In [6]:
W_conv1 = weight_variable([5, 5, 1, 32]) # 5 x 5 kernel, across an image with 1 channel to 32 channels
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
#h_pool1

In [7]:
W_conv2 = weight_variable([5, 5, 32, 64]) # 5 x 5 kernel, across an "image" with 32 channels to 64 channels
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)



### Stuff I've added

In [10]:
def tensor_dot(tensor1, tensor2, shape):
    dot = 0
    for i in range(shape[0]):
        for j in range(shape[1]):
            dot += tensor1[i,j] * tensor2[i,j]
    return dot

"""Realized can do this using tf operations reduce_sum and multiply"""



def anchor_box_dotprod(conv_feature, weight_box, anchor_point):
    w_shape = tf.shape(weight_box)
    
    image_box = conv_feature[anchor_point[0] - w_shape[0]/2 : anchor_point[0] + w_shape[0]/2 + 1, 
                         anchor_point[1] - w_shape[1]/2 : anchor_point[1] + w_shape[1]/2 + 1]
    ouptut = tf.reduce_sum(tf.multiply(weight_box, image_box))
    return output

def convolve2d(conv_feature, weight_box):
    f_0 = tf.shape(weight_box)[0]
    f_1 = tf.shape(weight_box)[1]
    paddings = [[(f_0 - 1)/2, (f_0 - 1)/2], [(f_1 - 1)/2, (f_1 - 1)/2]]
    image = tf.pad(conv_feature, paddings=paddings, mode='CONSTANT')
    
    start = [tf.shape(weight_box)[i]/2 for i in [0,1]]
    end = [tf.shape(conv_feature)[i] - starts[i] for i in [0,1]]
    #start_0, start_1 = f_0/2, f_1/2
    #end_0, end1 = tf.shape(conv_feature)[0] - f_0/2, tf.shape(conv_feature)[1] - f_1/2
    output_layer = [[anchor_box_dotprod(image,weight_box, [i,j]) for j in range(start[1],end[1] + 1)] for i in range(start[0], end[0] + 1)]
    return output_layer

"""Realized that tensorflow has an operation for this called depthwise_conv2d"""

def dw_conv2d(x,W):
    return tf.nn.depthwise_conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def bound_box(row,col,height,width,max_pool_shapes):
    x1, x2 = row, row + height
    y1, y2 = col, col + width
    for i in max_pool_shapes:
        x1 , x2 = (x1 - 1)  * i + 1, (x2 - 1) * i + 1
        y1, y2 = (y1 - 1 )* i + 1, (y2 - 1) * i + 1
    return tf.convert_to_tensor([x1,y1,x2,y2])

def regions_of_interest_old(region_scores,shapes1, shapes2, max_pool_shapes):
    batch_rois = []
    i = 0
    for height in shapes1:
        for width in shapes2:
            batch = region_scores[i]
            for image in batch:
                image_rois = []
                for row in range(tf.shape(image)[0]):
                    for col in range(tf.shape(image)[1]):
                        max_arg = image[row,col,tf.argmax(image[row,col,:])]
                        if(max_arg > 0):
                            image_rois.append(bound_box(row,col,height,width,max_pool_shapes))
                        """
                        isGreater = tf.greater(max_arg,0)
                        roi = tf.where(isGreater,bound_box(row,col,height,width,max_pool_shapes),None)
                        image_rois.append(roi)"""
                batch_rois.append(image_rois)
            i += 1
        rois_bathwise = tf.convert_to_tensor(batch_rois)
    return rois_batchwise


def regions_of_interest(region_scores,shapes1, shapes2, max_pool_shapes):
    
    region_rois = []
    sh = tf.shape(region_scores)[0]     
    """Hardcoded shapes because of weird error, must fix later"""
    #shoud be tf.shape(region_scores)[0] 
    for i in range(5):
        batch = region_scores[i]
        batch_rois = []
        height = shapes1[int(i / (len(shapes2)))]
        width = shapes2[i % len(shapes2)]
        images = tf.reduce_max(batch, axis = [3], keepdims = False)
        
        #should be tf.shape(images)[0]
        for j in range(minibatch_size):
            image = images[j]
            image_rois = []
            # should be tf.shape(image)[0]
            for row in range(7):
                #should be tf.shape(image)[1]
                for col in range(7):
                    toAppend = tf.cond(tf.equal(tf.maximum(image[row,col] , 0),0), 
                                       lambda : bound_box(0,0,0,0,[0]), 
                                       lambda : bound_box(row,col,height,width,max_pool_shapes))
                    image_rois.append(toAppend)
            batch_rois.append(image_rois)
        region_rois.append(batch_rois)
    final_rois = tf.concat([region_rois[i] for i in range(len(region_rois))], 1)
    return final_rois


In [11]:
"""
anchor_boxes
region_object_scores
tf.nn.max_pool(region_object_scores[0], ksize=[1, 1,1, 64],strides=[1, 1, 1, 64], padding='SAME')
tf.reduce_max(region_object_scores[0], axis = [3], keepdims = True)
"""
a = tf.constant(2, shape=[3,3])
b = tf.constant(3,shape=[3,3])

#region_object_scores[0]


anchor_boxes = [weight_variable([x,y,64, 1]) for x in [3,5,7] for y in [3,5,7]]
biases = [bias_variable([64]) for i in range(9)]   ###don't know if necessary


#not sure of whether to use h_conv2 or h_pool2

region_object_scores = tf.convert_to_tensor([tf.nn.relu(dw_conv2d(h_pool2, anchor_boxes[i]) + biases[i]) for i in range(9)])

rois = regions_of_interest(region_object_scores, [3,5,7] , [3,5,7], [2, 2])
with tf.Session() as sess:
    for i in range(tf.shape(a)[0].eval()):
        sess.run(tf.shape(a))

#region_object_scores

In [12]:
rois

<tf.Tensor 'concat:0' shape=(50, 245, 4) dtype=int32>

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024]) # This shape can be determined by plugging in something random * 64, and seing the resulting error. 
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
W_fc2 = weight_variable([1024, 256])
b_fc2 = bias_variable([256])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [ ]:
W_fc3 = weight_variable([256, 10])
b_fc3 = bias_variable([10])
y_out = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = y_out, labels = y_)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_, axis = 1), tf.argmax(y_out, axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        batch = mnist.train.next_batch(minibatch_size)
        training_inputs = batch[0].reshape([minibatch_size,28,28,1])
        training_labels = batch[1]
        if i % 100 == 0:
            print("epoch: {}".format(i))
            train_acc = accuracy.eval(feed_dict = {x: training_inputs, y_: training_labels, keep_prob : 1.0})
            print("training accuracy: {}".format(train_acc))
        sess.run([train_step], feed_dict = {x: training_inputs, y_: training_labels, keep_prob : keep})
    test_inputs = mnist.test.images.reshape([-1,28,28,1])
    test_labels = mnist.test.labels   
    test_acc = accuracy.eval(feed_dict = {x: test_inputs, y_: test_labels, keep_prob : 1.0})
    print("test accuracy: {}".format(test_acc))

In [ ]:
#Excess

"""i = tf.constant(1)
j = tf.constant(1)
fin = tf.Variable([0,0,0])
while_condition = lambda i,j,flag: i < 12

flag = 0
def body2(i,j):
    return [i,j+1]
def body(i,j, flag):
    #tf.Session().run(tf.while_loop(lambda i,j: j < 12, body2, (i,j)))
    if flag == 0:
        return [i , j + 1, flag]
    else:
        return [i + 1, j, flag - 1]
#fin = tf.while_loop(while_condition, body, (i,j, flag))
tf.Session().run(tf.while_loop(while_condition, body, (i,j, flag)))"""

"""W3_square = weight_variable([3,3])
W5_square = weight_variable([5,5])
W7_square = weight_variable([7,7])
W3_5 = weight_variable([3,5])
W5_3 = weight_variable([5,3])
W5_7 = weight_variable([5,7])
W3_7 = weight_variable([3,7])
W7_3 = weight_variable([7,3])
W7_5 = weight_variable([7,5])"""
#weight_boxes = [W3_square,W5_square, W7_square, W3_5, W5_3, W5_7, W3_7, W7_3, W7_5]